
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/GlennChia/50_038_cds_project/blob/master/notebooks/models/colab_gridsearcher.ipynb)

In [ ]:
#!pip install scikit-multilearn

In [1]:
# from google.colab import files
import numpy as np
import pandas as pd
import glob
import joblib

from sklearn.pipeline import Pipeline
# from sklearn.model_selection import GridSearchCV
import dask_ml.model_selection as dcv
from dask.diagnostics import ProgressBar
import joblib

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer

from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer


from sklearn.metrics import f1_score, hamming_loss, make_scorer, accuracy_score, precision_score, recall_score, classification_report, multilabel_confusion_matrix, confusion_matrix, roc_auc_score

from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.model_selection.measures import get_combination_wise_output_matrix
from skmultilearn.model_selection import iterative_train_test_split


In [2]:
DATA_DIR = "../../data/processed/"
INPUT_FILE_NAME = 'final_squash15_with_pos_ner_tm.parquet'
df = pd.read_parquet(DATA_DIR + INPUT_FILE_NAME)
df.head()

,speaker,headline,description,duration,tags,transcript,WC,clean_transcript,clean_transcript_string,sim_tags,squash15_tags,pos_sequence,ner_sequence,tm
0,Al Gore,Averting the climate crisis,With the same humor and humanity he exuded in ...,0:16:17,"cars,alternative energy,culture,politics,scien...","0:14\r\r\rThank you so much, Chris.\rAnd it's ...",2281.0,"b'[""thank"", ""chris"", ""truly"", ""great"", ""honor""...",thank chris truly great honor opportunity come...,"cars,solar system,energy,culture,politics,scie...","culture,politics,science,global issues,technology",VERB PROPN ADV ADJ NOUN NOUN VERB NOUN ADV ADV...,PERSON ORG ORG GPE LOC ORG PRODUCT GPE GPE PER...,"[0.04325945698517057, 0.0, 0.00142482934694180..."
1,Amy Smith,Simple designs to save a life,Fumes from indoor cooking fires kill more than...,0:15:06,"MacArthur grant,simplicity,industrial design,a...","0:11\r\r\rIn terms of invention,\rI'd like to ...",2687.0,"b'[""term"", ""invention"", ""like"", ""tell"", ""tale""...",term invention like tell tale favorite project...,"macarthur grant,simplicity,design,solar system...","design,global issues",NOUN NOUN SCONJ VERB PROPN ADJ NOUN VERB NOUN ...,GPE DATE CARDINAL DATE ORG PERSON LOC ORG GPE ...,"[0.013287880838036227, 0.0, 0.0, 0.00511725094..."
2,Ashraf Ghani,How to rebuild a broken state,Ashraf Ghani's passionate and powerful 10-minu...,0:18:45,"corruption,poverty,economics,investment,milita...","0:12\r\r\rA public, Dewey long ago observed,\r...",2506.0,"b'[""public"", ""dewey"", ""long"", ""ago"", ""observe""...",public dewey long ago observe constitute discu...,"corruption,inequality,science,investment,war,c...","science,culture,politics,global issues,business",ADJ PROPN ADV ADV VERB ADJ NOUN NOUN PROPN PRO...,DATE NORP ORDINAL DATE MONEY DATE DATE DATE EV...,"[0.0, 0.006699599134802422, 0.0, 0.00564851883..."
3,Burt Rutan,The real future of space exploration,"In this passionate talk, legendary spacecraft ...",0:19:37,"aircraft,flight,industrial design,NASA,rocket ...","0:11\r\r\rI want to start off by saying, Houst...",3092.0,"b'[""want"", ""start"", ""say"", ""houston"", ""problem...",want start say houston problem enter second ge...,"flight,design,nasa,science,invention,entrepren...","design,science,business",VERB NOUN VERB PROPN NOUN VERB ADJ NOUN NOUN N...,GPE ORDINAL ORG PERSON DATE DATE DATE TIME PER...,"[0.040282108339079505, 0.03732895646484358, 0...."
4,Chris Bangle,Great cars are great art,American designer Chris Bangle explains his ph...,0:20:04,"cars,industrial design,transportation,inventio...","0:12\r\r\rWhat I want to talk about is, as bac...",3781.0,"b'[""want"", ""talk"", ""background"", ""idea"", ""car""...",want talk background idea car art actually mea...,"cars,design,transportation,invention,technolog...","design,technology,business,science",VERB NOUN NOUN NOUN NOUN NOUN ADV ADJ NOUN NOU...,PERSON PRODUCT ORG ORG PERSON PERSON PERSON OR...,"[0.08049208168957463, 0.0, 0.0, 0.008031187136..."


In [3]:
df = df.dropna(subset=['squash15_tags'])
df = df.reset_index(drop=True)
X = df['clean_transcript_string']
labels = df[['squash15_tags']]

In [4]:
y = []
for index, row in labels.iterrows():
    y.append(set(row['squash15_tags'].split(',')))
    
mlb = MultiLabelBinarizer()
encoded_y = mlb.fit_transform(y)

In [5]:
X_tm = df['tm']

In [6]:
X_joined = pd.concat([df[['clean_transcript_string']], pd.DataFrame(np.array(list(X_tm)))], axis=1).values

In [7]:
from skmultilearn.model_selection import iterative_train_test_split

X_train, y_train, X_test, y_test = iterative_train_test_split(X_joined, encoded_y, test_size = 0.2)
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

In [9]:
tags = [tag for tag in mlb.inverse_transform(np.ones(shape=(1, 15)))[0]]
print(tags)
tags.index('technology')

['biodiversity', 'biomechanics', 'business', 'communication', 'culture', 'design', 'entertainment', 'future', 'global issues', 'history', 'humanity', 'media', 'politics', 'science', 'technology']


14

In [10]:
# param_range = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
param_range = [1, 2, 3, 4, 5]
param_range_lr = [1.0, 0.5, 0.1]

# Set params, comment out as see fit

vectorizer = Pipeline(steps=[
    ('vec', TfidfVectorizer()),
])

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

preprocessor = ColumnTransformer(
    transformers=[
        ('vectorizer', vectorizer, 0),
        ('num', numeric_transformer, list(range(1, 16))),
    ])

lr_params = {
    'clf': [LogisticRegression()],
        'clf__penalty': ['l1', 'l2'],
        'clf__C': [0.01, 0.1, 1, 10, 50, 100, 500],
        'clf__solver': ['liblinear'],
        'clf__class_weight': ['balanced'],
}

svc_params = {
    'clf': [SVC()],
        'clf__kernel': ['linear', 'rbf'],
        'clf__C': param_range, # np.logspace(-1, 2, 10),
        'clf__gamma': ['auto'], # np.logspace(-1, 1, 10)
        'clf__probability': [True],
        'clf__class_weight': ['balanced'],
}

rf_params = {
    'clf': [RandomForestClassifier()],
        'clf__criterion': ['gini', 'entropy'],
        'clf__min_samples_leaf': [2, 4, 8],
#         'clf__max_depth': np.linspace(10, 50, num = 5),
        'clf__min_samples_split': [2, 5, 10],
        'clf__n_estimators': [60, 100, 200, 500],
        'clf__class_weight': ['balanced'],
}

mnb_params = {
    'clf': [MultinomialNB()],
        'clf__alpha': [0.7, 1.0],
}

## Stack params
parameters = [
    lr_params,
#     svc_params,
    rf_params,
#     mnb_params
]

per_tag_pipe = Pipeline([('processor', preprocessor),
                  ('clf', LogisticRegression())], verbose=True)

# scoring = make_scorer(hamming_loss)
scoring = 'f1'
# scoring = 'f1_micro'
# scoring = 'balanced_accuracy'
# scoring = 'precision'
folds = 4
njobs = -1

per_tag_model = dcv.GridSearchCV(per_tag_pipe, parameters, scoring=scoring, cv=folds, n_jobs=njobs)

In [11]:
%%time
for index in range(len(tags)):
    print(f"Processing {tags[index]}")
    with ProgressBar():
        per_tag_model.fit(X_train, y_train[:, index])
    filename = f'best_{tags[index]}_model.joblib'
    joblib.dump(per_tag_model, filename)

Processing biodiversity
[###                                     ] | 7% Completed | 13.9s

C:\Users\kayes\Anaconda3\envs\cds\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[###                                     ] | 8% Completed | 15.2s

C:\Users\kayes\Anaconda3\envs\cds\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\kayes\Anaconda3\envs\cds\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[###                                     ] | 9% Completed | 18.6s

...

[######################################  ] | 95% Completed |  5min  9.8s

C:\Users\kayes\Anaconda3\envs\cds\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[######################################  ] | 95% Completed |  5min 11.2s

C:\Users\kayes\Anaconda3\envs\cds\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[########################################] | 100% Completed |  5min 25.7s
[                                        ] | 0% Completed |  1.4s[Pipeline] ......... (step 1 of 2) Processing processor, total=   1.4s
[                                        ] | 0% Completed |  2.6s[Pipeline] ............... (step 2 of 2) Processing clf, total=   1.2s
[########################################] | 100% Completed |  2.7s
Processing science
[########################################] | 100% Completed |  6min 36.9s
[                                        ] | 0% Completed |  1.3s[Pipeline] ......... (step 1 of 2) Processing processor, total=   1.3s
[                                        ] | 0% Completed | 10.9s[Pipeline] ............... (step 2 of 2) Processing clf, total=   9.6s
[########################################] | 100% Completed | 11.1s
Processing technology
[########################################] | 100% Completed |  6min 26.5s
[                                        ] | 0% Completed

In [12]:
# TODO: 
# 1. Check if TfidfTransformer use_idf=False is the same as Countvectorizer? or there are other metrics to suppress
# 2. Get scoring function to work, hamming? -- kinda done
# 3. Balanced class labels
# 4. Set better param ranges
# 5. Remove vectorizer step once we decide on which is better, then use sparse csr and hopefully it trains faster

# param_range = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
param_range = [1, 2, 3, 4, 5]
param_range_lr = [1.0, 0.5, 0.1]

# Set params, comment out as see fit

vectorizer = Pipeline(steps=[
    ('vec', TfidfVectorizer()),
])

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

preprocessor = ColumnTransformer(
    transformers=[
        ('vectorizer', vectorizer, 0),
        ('num', numeric_transformer, list(range(1, 16))),
    ])


lr_params = {
    'clf__classifier': [LogisticRegression()],
        'clf__classifier__penalty': ['l1', 'l2'],
        'clf__classifier__C': [0.01, 0.1, 1, 10, 50, 100, 500],
        'clf__classifier__solver': ['liblinear'],
        'clf__classifier__class_weight': ['balanced'],
}

svc_params = {
    'clf__classifier': [SVC()],
        'clf__classifier__kernel': ['linear', 'rbf'],
        'clf__classifier__C': param_range, # np.logspace(-1, 2, 10),
        'clf__classifier__gamma': ['auto'], # np.logspace(-1, 1, 10)
        'clf__classifier__probability': [True],
        'clf__classifier__class_weight': ['balanced'],
}

rf_params = {
    'clf__classifier': [RandomForestClassifier()],
        'clf__classifier__criterion': ['gini', 'entropy'],
        'clf__classifier__min_samples_leaf': param_range,
        'clf__classifier__max_depth': param_range,
        'clf__classifier__min_samples_split': param_range[1:],
        'clf__classifier__n_estimators': [10],
        'clf__classifier__class_weight': ['balanced'],
}

mnb_params = {
    'clf__classifier': [MultinomialNB()],
        'clf__classifier__alpha': [0.7, 1.0, 1.5],
}

## Stack params
parameters = [
    lr_params,
#     svc_params,
#     rf_params,
#     mnb_params
]


br_pipeline = Pipeline([('processor', preprocessor),
                        ('clf', BinaryRelevance()),
                       ]
                      )


# Gridsearch settings
# scoring = make_scorer(f1_score, average='micro') # possible scorings 'f1_micro' 'f1_macro'
# scoring = 'f1_micro'
# scoring = make_scorer(hamming_loss)
# scoring = 'neg_log_loss'
scoring = 'f1_micro'
folds = 4
njobs = -1

br_model = dcv.GridSearchCV(br_pipeline, parameters, scoring=scoring, cv=folds, n_jobs=njobs)

In [13]:
with ProgressBar():
    br_model.fit(X_train, y_train)
print(br_model.best_params_, br_model.best_score_)
pd.DataFrame(br_model.cv_results_)
filename = f'best_br_model.joblib'
joblib.dump(br_model, filename)

[########################################] | 100% Completed |  4min 44.1s
[########################################] | 100% Completed | 23.3s
{'clf__classifier': LogisticRegression(C=50, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False), 'clf__classifier__C': 50, 'clf__classifier__class_weight': 'balanced', 'clf__classifier__penalty': 'l2', 'clf__classifier__solver': 'liblinear'} 0.5641318118401925


['best_br_model.joblib']

In [14]:
pd.set_option('display.max_colwidth', 200)

In [15]:
tags = [tag for tag in mlb.inverse_transform(np.ones(shape=(1, 15)))[0]]
print(tags)
tags.index('technology')

['biodiversity', 'biomechanics', 'business', 'communication', 'culture', 'design', 'entertainment', 'future', 'global issues', 'history', 'humanity', 'media', 'politics', 'science', 'technology']


14

In [16]:
(y_pred.A > 0.5).astype(int)

NameError: name 'y_pred' is not defined

In [17]:
%%time
br_model = joblib.load('best_br_model.joblib')
y_pred = br_model.predict_proba(X_test)
y_pred_new = (y_pred > 0.5).astype(int)
br_model_results = {}

for index in range(len(tags)):
#     print(f"Processing {tags[index]}")
    prediction = y_pred_new[:, index].A
    
    if (sum(y_train[:, index]) / len(y_train)) < 0.5:
        baseline_prediction = np.zeros(len(y_test)).astype(int)
    else:
        baseline_prediction = np.ones(len(y_test)).astype(int)
    true_y = y_test[:, index]
    
    br_model_results[tags[index]] = {
        'precision': precision_score(true_y, prediction, average="binary"),
        'recall': recall_score(true_y, prediction, average="binary"),
        'f1_score': f1_score(true_y, prediction, average="binary"),
        'baseline_f1': f1_score(true_y, np.ones(len(y_test)).astype(int), average="binary"),
        'accuracy': accuracy_score(true_y, prediction),
        'baseline_accuracy': accuracy_score(true_y, baseline_prediction),
        'conf_matrix': confusion_matrix(true_y, prediction),
        'roc_auc': roc_auc_score(true_y, y_pred[:, index].A),
        'best_model': br_model.best_params_
        
    }
    print('--------------------------')

# print(pd.DataFrame())
br_results = pd.DataFrame(br_model_results).transpose()
br_results

--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
Wall time: 3.64 s


,precision,recall,f1_score,baseline_f1,accuracy,baseline_accuracy,conf_matrix,roc_auc,best_model
biodiversity,0.488889,0.488889,0.488889,0.169173,0.905544,0.907598,"[[419, 23], [23, 22]]",0.890096,"{'clf__classifier': LogisticRegression(C=50, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_..."
biomechanics,0.553191,0.590909,0.571429,0.165725,0.919918,0.909651,"[[422, 21], [18, 26]]",0.912323,"{'clf__classifier': LogisticRegression(C=50, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_..."
business,0.515625,0.471429,0.492537,0.251346,0.86037,0.856263,"[[386, 31], [37, 33]]",0.835834,"{'clf__classifier': LogisticRegression(C=50, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_..."
communication,0.391304,0.204545,0.268657,0.165725,0.899384,0.909651,"[[429, 14], [35, 9]]",0.80551,"{'clf__classifier': LogisticRegression(C=50, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_..."
culture,0.618644,0.579365,0.598361,0.682003,0.597536,0.482546,"[[145, 90], [106, 146]]",0.639514,"{'clf__classifier': LogisticRegression(C=50, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_..."
design,0.633663,0.64,0.636816,0.340716,0.850103,0.794661,"[[350, 37], [36, 64]]",0.880646,"{'clf__classifier': LogisticRegression(C=50, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_..."
entertainment,0.553191,0.45614,0.5,0.209559,0.893224,0.882957,"[[409, 21], [31, 26]]",0.86022,"{'clf__classifier': LogisticRegression(C=50, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_..."
future,0.3,0.146341,0.196721,0.155303,0.899384,0.915811,"[[432, 14], [35, 6]]",0.730285,"{'clf__classifier': LogisticRegression(C=50, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_..."
global issues,0.654135,0.635036,0.644444,0.439103,0.802875,0.718686,"[[304, 46], [50, 87]]",0.827654,"{'clf__classifier': LogisticRegression(C=50, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_..."
history,0.285714,0.133333,0.181818,0.311958,0.778234,0.815195,"[[367, 30], [78, 12]]",0.621942,"{'clf__classifier': LogisticRegression(C=50, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_..."


In [18]:
pd.set_option('display.max_colwidth', 250)

In [19]:
%%time
all_tags_model_results = {}
for index in range(len(tags)):
    all_tags_model = joblib.load(f"best_{tags[index]}_model.joblib")
    print(f"Processing {tags[index]}")
    pred_prob = all_tags_model.predict_proba(X_test)
    prediction = all_tags_model.predict(X_test)
    
    if (sum(y_train[:, index]) / len(y_train)) < 0.5:
        baseline_prediction = np.zeros(len(y_test)).astype(int)
    else:
        baseline_prediction = np.ones(len(y_test)).astype(int)
        
    true_y = y_test[:, index]
    
    all_tags_model_results[tags[index]] = {
        'precision': precision_score(true_y, prediction, average="binary"),
        'recall': recall_score(true_y, prediction, average="binary"),
        'f1_score': f1_score(true_y, prediction, average="binary"),
        'baseline_f1_score': f1_score(true_y, np.ones(len(y_test)).astype(int), average="binary"),
        'accuracy': accuracy_score(true_y, prediction),
        'baseline_accuracy': accuracy_score(true_y, baseline_prediction),
        'conf_matrix': confusion_matrix(true_y, prediction),
        'roc_auc': roc_auc_score(true_y, pred_prob[:,1] ),
        'best_model': all_tags_model.best_params_
        
    }
    print('--------------------------')
per_tag_results = pd.DataFrame(all_tags_model_results).transpose()

Processing biodiversity
--------------------------
Processing biomechanics
--------------------------
Processing business
--------------------------
Processing communication
--------------------------
Processing culture
--------------------------
Processing design
--------------------------
Processing entertainment
--------------------------
Processing future
--------------------------
Processing global issues
--------------------------
Processing history
--------------------------
Processing humanity
--------------------------
Processing media
--------------------------
Processing politics
--------------------------
Processing science
--------------------------
Processing technology
--------------------------
Wall time: 28.4 s


In [20]:
per_tag_results

,precision,recall,f1_score,baseline_f1_score,accuracy,baseline_accuracy,conf_matrix,roc_auc,best_model
biodiversity,0.378049,0.688889,0.488189,0.169173,0.86653,0.907598,"[[391, 51], [14, 31]]",0.89814,"{'clf': LogisticRegression(C=1, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='warn', n_jobs=None, penalty='l1', ran..."
biomechanics,0.459459,0.772727,0.576271,0.165725,0.897331,0.909651,"[[403, 40], [10, 34]]",0.916992,"{'clf': LogisticRegression(C=1, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='warn', n_jobs=None, penalty='l2', ran..."
business,0.5,0.5,0.5,0.251346,0.856263,0.856263,"[[382, 35], [35, 35]]",0.840562,"{'clf': LogisticRegression(C=10, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='warn', n_jobs=None, penalty='l2', ra..."
communication,0.27027,0.454545,0.338983,0.165725,0.839836,0.909651,"[[389, 54], [24, 20]]",0.793351,"{'clf': LogisticRegression(C=1, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='warn', n_jobs=None, penalty='l2', ran..."
culture,0.625,0.694444,0.657895,0.682003,0.626283,0.482546,"[[130, 105], [77, 175]]",0.657751,"{'clf': (DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, ..."
design,0.5,0.76,0.603175,0.340716,0.794661,0.794661,"[[311, 76], [24, 76]]",0.852171,"{'clf': LogisticRegression(C=1, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='warn', n_jobs=None, penalty='l1', ran..."
entertainment,0.482143,0.473684,0.477876,0.209559,0.87885,0.882957,"[[401, 29], [30, 27]]",0.858711,"{'clf': LogisticRegression(C=10, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='warn', n_jobs=None, penalty='l2', ra..."
future,0.210526,0.390244,0.273504,0.155303,0.825462,0.915811,"[[386, 60], [25, 16]]",0.706606,"{'clf': LogisticRegression(C=1, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='warn', n_jobs=None, penalty='l2', ran..."
global issues,0.597484,0.693431,0.641892,0.439103,0.782341,0.718686,"[[286, 64], [42, 95]]",0.837289,"{'clf': LogisticRegression(C=1, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='warn', n_jobs=None, penalty='l2', ran..."
history,0.246753,0.422222,0.311475,0.311958,0.655031,0.815195,"[[281, 116], [52, 38]]",0.569242,"{'clf': LogisticRegression(C=0.01, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='warn', n_jobs=None, penalty='l1', ..."


In [21]:
print('Binary Relevance, single base model')
br_results

Binary Relevance, single base model


,precision,recall,f1_score,baseline_f1,accuracy,baseline_accuracy,conf_matrix,roc_auc,best_model
biodiversity,0.488889,0.488889,0.488889,0.169173,0.905544,0.907598,"[[419, 23], [23, 22]]",0.890096,"{'clf__classifier': LogisticRegression(C=50, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='warn', n_jobs=None, penalty='l2', ..."
biomechanics,0.553191,0.590909,0.571429,0.165725,0.919918,0.909651,"[[422, 21], [18, 26]]",0.912323,"{'clf__classifier': LogisticRegression(C=50, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='warn', n_jobs=None, penalty='l2', ..."
business,0.515625,0.471429,0.492537,0.251346,0.86037,0.856263,"[[386, 31], [37, 33]]",0.835834,"{'clf__classifier': LogisticRegression(C=50, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='warn', n_jobs=None, penalty='l2', ..."
communication,0.391304,0.204545,0.268657,0.165725,0.899384,0.909651,"[[429, 14], [35, 9]]",0.80551,"{'clf__classifier': LogisticRegression(C=50, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='warn', n_jobs=None, penalty='l2', ..."
culture,0.618644,0.579365,0.598361,0.682003,0.597536,0.482546,"[[145, 90], [106, 146]]",0.639514,"{'clf__classifier': LogisticRegression(C=50, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='warn', n_jobs=None, penalty='l2', ..."
design,0.633663,0.64,0.636816,0.340716,0.850103,0.794661,"[[350, 37], [36, 64]]",0.880646,"{'clf__classifier': LogisticRegression(C=50, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='warn', n_jobs=None, penalty='l2', ..."
entertainment,0.553191,0.45614,0.5,0.209559,0.893224,0.882957,"[[409, 21], [31, 26]]",0.86022,"{'clf__classifier': LogisticRegression(C=50, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='warn', n_jobs=None, penalty='l2', ..."
future,0.3,0.146341,0.196721,0.155303,0.899384,0.915811,"[[432, 14], [35, 6]]",0.730285,"{'clf__classifier': LogisticRegression(C=50, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='warn', n_jobs=None, penalty='l2', ..."
global issues,0.654135,0.635036,0.644444,0.439103,0.802875,0.718686,"[[304, 46], [50, 87]]",0.827654,"{'clf__classifier': LogisticRegression(C=50, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='warn', n_jobs=None, penalty='l2', ..."
history,0.285714,0.133333,0.181818,0.311958,0.778234,0.815195,"[[367, 30], [78, 12]]",0.621942,"{'clf__classifier': LogisticRegression(C=50, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='warn', n_jobs=None, penalty='l2', ..."


In [22]:
print('Binary relevance, individual models')
per_tag_results

Binary relevance, individual models


,precision,recall,f1_score,baseline_f1_score,accuracy,baseline_accuracy,conf_matrix,roc_auc,best_model
biodiversity,0.378049,0.688889,0.488189,0.169173,0.86653,0.907598,"[[391, 51], [14, 31]]",0.89814,"{'clf': LogisticRegression(C=1, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='warn', n_jobs=None, penalty='l1', ran..."
biomechanics,0.459459,0.772727,0.576271,0.165725,0.897331,0.909651,"[[403, 40], [10, 34]]",0.916992,"{'clf': LogisticRegression(C=1, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='warn', n_jobs=None, penalty='l2', ran..."
business,0.5,0.5,0.5,0.251346,0.856263,0.856263,"[[382, 35], [35, 35]]",0.840562,"{'clf': LogisticRegression(C=10, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='warn', n_jobs=None, penalty='l2', ra..."
communication,0.27027,0.454545,0.338983,0.165725,0.839836,0.909651,"[[389, 54], [24, 20]]",0.793351,"{'clf': LogisticRegression(C=1, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='warn', n_jobs=None, penalty='l2', ran..."
culture,0.625,0.694444,0.657895,0.682003,0.626283,0.482546,"[[130, 105], [77, 175]]",0.657751,"{'clf': (DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, ..."
design,0.5,0.76,0.603175,0.340716,0.794661,0.794661,"[[311, 76], [24, 76]]",0.852171,"{'clf': LogisticRegression(C=1, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='warn', n_jobs=None, penalty='l1', ran..."
entertainment,0.482143,0.473684,0.477876,0.209559,0.87885,0.882957,"[[401, 29], [30, 27]]",0.858711,"{'clf': LogisticRegression(C=10, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='warn', n_jobs=None, penalty='l2', ra..."
future,0.210526,0.390244,0.273504,0.155303,0.825462,0.915811,"[[386, 60], [25, 16]]",0.706606,"{'clf': LogisticRegression(C=1, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='warn', n_jobs=None, penalty='l2', ran..."
global issues,0.597484,0.693431,0.641892,0.439103,0.782341,0.718686,"[[286, 64], [42, 95]]",0.837289,"{'clf': LogisticRegression(C=1, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='warn', n_jobs=None, penalty='l2', ran..."
history,0.246753,0.422222,0.311475,0.311958,0.655031,0.815195,"[[281, 116], [52, 38]]",0.569242,"{'clf': LogisticRegression(C=0.01, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='warn', n_jobs=None, penalty='l1', ..."
